In [5]:
'''
啟動伺服器樣板
'''
from flask import Flask,request,abort
from linebot import LineBotApi,WebhookHandler
from linebot.exceptions import InvalidSignatureError
import json

secret_file = json.load(open('line_bot_secret_key.json','r'))
server_url = secret_file.get('server_url')
app = Flask(__name__,static_url_path='/image_trpg_elevator',static_folder='../image_trpg_elevator/')

line_bot_api = LineBotApi(secret_file.get("channel_access_token"))
handler = WebhookHandler(secret_file.get('secret_key'))

@app.route('/',methods = ['POST'])
def callback():
    signature = request.headers['X-Link-Signature']
    body = request.get_data(as_text=True)
    app.logger.info('Request body:' + body)

    try:
        handler.handle(body,signature)
    except:
        abort(400)
    return "OK"

In [6]:
from linebot.models import TextSendMessage,ImageSendMessage,MessageEvent,TextMessage
# 消息製作
reply_message_list = [
# ImageSendMessage(original_content_url='https://{}/warning.png'.format(server_url)),
    TextSendMessage(text='本遊戲含有恐怖及克蘇魯神話成分，請斟酌遊玩')
]

In [7]:
from linebot.models.events import FollowEvent
import requests

# 這裡是在做發消息及取個資的動作

@handler.add(FollowEvent)
def reply_user_and_get_user_id(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    with open('../users_profile.txt','a') as us_file:
        us_file.write(json.dumps(vars(user_profile),sort_keys=True))
        us_file.write('\r\n')

    # 綁定圖文選單
    LineRichMenuMainID = secret_file.get('rich_menu_id')
    linkResult = line_bot_api.link_rich_menu_to_user(secret_file["self_user_id"],LineRichMenuMainID)

    # 關注回應
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [8]:
if __name__ =='__main__':
    app.run(host = '0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
[2019-11-23 11:57:14,362] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\max36\Anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\max36\Anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\max36\Anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\max36\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\max36\Anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\max36\Anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-5-bf9acd